## Source Information
---
**Author**:

**Last Updated Date**: October 01, 2024 by Gloria Seo

**Resources**: https://medium.com/analytics-vidhya/sequence-model-time-series-prediction-using-tensorflow-2-0-665257beb25f

---

## Goal
This Jupyter notebook demonstrates how to build an image classification model using TensorFlow and Keras.
The goal is to classify images of flowers into five categories: daisies, dandelions, roses, sunflowers, and tulips. 

## Required Modules for the Jupyter Notebook

Before running the notebook, make sure to load the following modules.

**Module: tensorflow, numpy, PIL, matplotlib,keras,layers, Sequential, pathlib** 

In [2]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████            | 299.7 MB 117.1 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 479.6 MB 41 kB/s 
     |████████████████████████████████| 277 kB 152.2 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 144.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 139.8 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 706 kB/s s eta 0:00:01
     |████████████████████████████████| 6.0 MB 50.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 103.0 MB/s eta 0:00:01
     |████████████████████████████████| 24.5 MB 134.3 MB/s eta 0:00:01
     |████████████████████████████████| 440 kB 149.2 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 140.4 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 29 kB/s /s eta 0:00:01
     |████████████████████████████████| 5.6 MB 143.3 MB/s eta 0:00:01
     |████████████████████████████████| 210 kB 149.2 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 74.9 MB/s eta 0:00:01
     |████████████████████████████

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib

ModuleNotFoundError: No module named 'tensorflow'

## Loading the Data Set
The dataset is a collection of flower photos available online. We use TensorFlow’s utility function to download and extract it.

The dataset used is a collection of flower images, which is freely available online. This notebook uses a TensorFlow utility function to download and extract these images. The dataset includes 3,670 images of the five flower categories.

In [ ]:
# Image Classification Example for Comet, Stratus, Expanse

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)
data_dir

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

It means that the total number of images in the dataset is 3670.

In [ ]:
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
batch_size = 32
img_height = 180
img_width = 180
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
num_classes = 5

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In Tensorflow, you have to compile the model graph before you begin training. This speeds up the runtime.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

## Finally, we actually get to train something!

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

### Well we've trained... but now what? Did I do good?!
The answer is no. Run the below cell, and look at the graphs. The validation loss increases, while the training loss continues to decrease. This means your model is memorizing the training set, and isn't performing well on "real world" data (data it has never seen before). We're going to fix that by flipping and rotating the images in our dataset to artifically increase the number of pictures we have.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

We're also going to introduce a dropout layer, which randomly removes some of the output layers. It does this because one of these layers may in fact be misleading, making us think our model did well when in reality this is a local minima of the function, not a global one. By doing this randomly we reduce this situation occuring enough to get a better model. The strong will survive, so even if we remove a good output its ok, its better than having a lot of misleading ones.

In [ ]:
model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

In [ ]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

After running the below cell, notice that the validation error decreases along with the training error.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('raining and Validation Loss')
plt.show()

In [ ]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = keras.preprocessing.image.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

## Submit Ticket
If you find anything that needs to be changed, edited, or if you would like to provide feedback or contribute to the notebook, please submit a ticket by contacting us at:

Email: consult@sdsc.edu

We appreciate your input and will review your suggestions promptly!